In [2]:
pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 24.7 MB/s eta 0:00:00


In [3]:
from google.colab import drive
import pandas as pd
from underthesea import  word_tokenize

from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble

import pandas, xgboost, numpy, textblob, string

from keras.preprocessing import text, sequence
from keras import layers, models, optimizers

import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from gensim.utils import simple_preprocess
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

import torch.nn as nn
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader

from transformers import get_linear_schedule_with_warmup, AutoTokenizer, AutoModel, logging

import warnings
warnings.filterwarnings("ignore")

logging.set_verbosity_error()



In [4]:
drive.mount('/drive')
path = '/drive/MyDrive/Colab Notebooks/Data/Data_train_1.xlsx'

df = pd.read_excel(path)

print(df);

rows = len(df)
print(rows)



Mounted at /drive


FileNotFoundError: [Errno 2] No such file or directory: '/drive/MyDrive/Colab Notebooks/Data/Data_train_1.xlsx'

In [ ]:
# for i in range(rows):
#   df['WORDS'][i] = word_tokenize(df['WORDS'][i], format = 'text')

print(df)

In [ ]:


#split train set and validation test from df


# print(train_x.info())
# print(valid_x.info())


In [ ]:
def train_model(classifier, feature_vector_train, label, feature_vector_valid, is_neural_net=False):
      # fit the training dataset on the classifier
      classifier.fit(feature_vector_train, label)

      # predict the labels on validation dataset
      predictions = classifier.predict(feature_vector_valid)

      if is_neural_net:
          predictions = predictions.argmax(axis=-1)

      return metrics.accuracy_score(predictions, valid_y)


result = open("result.txt","w")
loop = 100
average_acc_NB = 0
average_acc_SVM = 0
average_acc_RF = 0

for i in range(loop):
  print("-"*30)
  print(f"Loop {i+1}:")
  result.write(f"\nLoop {i+1}:\n\n")
  train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df['WORDS'], df['Label'])

  # Feature extraction using ngram level TF-IDF
  tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
  tfidf_vect_ngram.fit(df['WORDS'])
  xtrain_tfidf_ngram =  tfidf_vect_ngram.transform(train_x)
  xvalid_tfidf_ngram =  tfidf_vect_ngram.transform(valid_x)

  #Naive Bayes
  accuracy = train_model(naive_bayes.MultinomialNB(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
  average_acc_NB += accuracy
  print("NB, N-Gram Vectors: ", accuracy)
  result.write(f"NB, N-Gram Vectors: {accuracy}\n\n")

  #SVM
  accuracy = train_model(svm.SVC(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
  average_acc_SVM += accuracy
  print("SVM, N-Gram Vectors: ", accuracy)
  result.write(f"SVM, N-Gram Vectors: {accuracy}\n\n")

  #Random Forest
  accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_tfidf_ngram, train_y, xvalid_tfidf_ngram)
  average_acc_RF += accuracy
  print("RF, N-Gram Vectors: ", accuracy)
  result.write(f"RF, N-Gram Vectors: {accuracy}\n\n")

print(f"Average accuracy of NB: {average_acc_NB/loop}")
print(f"Average accuracy of SVM: {average_acc_SVM/loop}")
print(f"Average accuracy of RF: {average_acc_RF/loop}")

In [ ]:
print(df)

In [ ]:
#using PhoBert
def seed_everything(seed_value):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)

    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value)
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = True

seed_everything(86)

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
EPOCHS = 6
N_SPLITS = 5

skf = StratifiedKFold(n_splits=N_SPLITS)
for fold, (_, val_) in enumerate(skf.split(X=df, y=df.Label)):
    df.loc[val_, "kfold"] = fold

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

class TypeClassificationDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=120):
        self.df = df
        self.max_len = max_len
        self.tokenizer = tokenizer

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        """
        To customize dataset, inherit from Dataset class and implement
        __len__ & __getitem__
        __getitem__ should return
            data:
                input_ids
                attention_masks
                text
                targets
        """
        row = self.df.iloc[index]
        text, label = self.get_input_data(row)

        # Encode_plus will:
        # (1) split text into token
        # (2) Add the '[CLS]' and '[SEP]' token to the start and end
        # (3) Truncate/Pad sentence to max length
        # (4) Map token to their IDS
        # (5) Create attention mask
        # (6) Return a dictionary of outputs
        encoding = self.tokenizer.encode_plus(
            text,
            truncation=True,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors='pt',
        )

        return {
            'text': text,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_masks': encoding['attention_mask'].flatten(),
            'targets': torch.tensor(label, dtype=torch.long),
        }

    def get_input_data(self, row):
        # Preprocessing: {remove icon, special character, lower}
        text = row['WORDS']
        text = ' '.join(simple_preprocess(text))
        label = row['Label']

        return text, label

In [ ]:
class TypeClassifier(nn.Module):
    def __init__(self, n_classes):
        super(TypeClassifier, self).__init__()
        self.bert = AutoModel.from_pretrained("vinai/phobert-base")
        self.drop = nn.Dropout(p=0.3)
        self.fc = nn.Linear(self.bert.config.hidden_size, n_classes)
        nn.init.normal_(self.fc.weight, std=0.02)
        nn.init.normal_(self.fc.bias, 0)

    def forward(self, input_ids, attention_mask):
        last_hidden_state, output = self.bert(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=False # Dropout will errors if without this
        )

        x = self.drop(output)
        x = self.fc(x)
        return x

In [ ]:
def train(model, criterion, optimizer, train_loader):
    model.train()
    losses = []
    correct = 0

    for data in train_loader:
        input_ids = data['input_ids'].to(device)
        attention_mask = data['attention_masks'].to(device)
        targets = data['targets'].to(device)

        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        loss = criterion(outputs, targets)
        _, pred = torch.max(outputs, dim=1)

        correct += torch.sum(pred == targets)
        losses.append(loss.item())
        loss.backward()
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        lr_scheduler.step()

    print(f'Train Accuracy: {correct.double()/len(train_loader.dataset)} Loss: {np.mean(losses)}')

def eval(test_data = False):
    model.eval()
    losses = []
    correct = 0

    with torch.no_grad():
        data_loader = test_loader if test_data else valid_loader
        for data in data_loader:
            input_ids = data['input_ids'].to(device)
            attention_mask = data['attention_masks'].to(device)
            targets = data['targets'].to(device)

            outputs = model(
                input_ids=input_ids,
                attention_mask=attention_mask
            )

            _, pred = torch.max(outputs, dim=1)

            loss = criterion(outputs, targets)
            correct += torch.sum(pred == targets)
            losses.append(loss.item())

    if test_data:
        print(f'Test Accuracy: {correct.double()/len(test_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(test_loader.dataset)
    else:
        print(f'Valid Accuracy: {correct.double()/len(valid_loader.dataset)} Loss: {np.mean(losses)}')
        return correct.double()/len(valid_loader.dataset)

In [ ]:
def prepare_loaders(df, fold):

    df_train = df[df.kfold != fold].reset_index(drop=True)
    df_valid = df[df.kfold == fold].reset_index(drop=True)

    train_dataset = TypeClassificationDataset(df_train, tokenizer, max_len=120)
    valid_dataset = TypeClassificationDataset(df_valid, tokenizer, max_len=120)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
    valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True, num_workers=2)

    return train_loader, valid_loader

In [ ]:
# for fold in range(skf.n_splits):
#     print(f'\n-----------Fold: {fold+1} ------------------')
#     train_loader, valid_loader = prepare_loaders(df, fold=fold)
#     model = TypeClassifier(n_classes=7).to(device)
#     criterion = nn.CrossEntropyLoss()
#     # Recommendation by BERT: lr: 5e-5, 2e-5, 3e-5
#     # Batchsize: 16, 32
#     optimizer = AdamW(model.parameters(), lr=2e-5)

#     lr_scheduler = get_linear_schedule_with_warmup(
#                 optimizer,
#                 num_warmup_steps=0,
#                 num_training_steps=len(train_loader)*EPOCHS
#             )
#     best_acc = 0
#     for epoch in range(EPOCHS):
#         print(f'Epoch {epoch+1}/{EPOCHS}')
#         print('-'*30)

#         train(model, criterion, optimizer, train_loader)
#         val_acc = eval()

#         if val_acc > best_acc:
#             torch.save(model.state_dict(), f'/drive/MyDrive/Colab Notebooks/Data/phobert_fold{fold+1}.pth')
#             best_acc = val_acc